Managed tables - completely managed by UC (create, delete, update, resolve conflicts if any)

External tables - not managed by UC (no error thrown even if we attempt to create multiple tables with same name). 

In [0]:
%sql
drop catalog if exists catalog_databricks cascade;
-- drop schema if exists catalog_databricks.schema_databricks cascade;
-- drop table if exists catalog_databricks.schema_databricks.tbl_databricks;
-- drop volume if exists catalog_databricks.schema_databricks.volume_databricks;

In [0]:
%sql
create catalog if not exists catalog_databricks 
MANAGED LOCATION 's3://s3bucket-databrickslearning/catalog_databricks/';

create schema if not exists catalog_databricks.schema_databricks 
MANAGED LOCATION 's3://s3bucket-databrickslearning/catalog_databricks/schema_databricks/';

create table if not exists catalog_databricks.schema_databricks.table_databricks_managed;

create table if not exists catalog_databricks.schema_databricks.table_databricks_external
using delta
LOCATION 's3://s3bucket-databrickslearning/catalog_databricks/schema_databricks/table_databricks_external/';

create volume if not exists catalog_databricks.schema_databricks.volume_databricks_managed;

create external volume if not exists catalog_databricks.schema_databricks.volume_databricks_external
LOCATION 's3://s3bucket-databrickslearning/volume_databricks_external';


In [0]:
%python
display(spark.sql(f"describe catalog extended catalog_databricks"))
display(spark.sql(f"describe schema extended catalog_databricks.schema_databricks"))
display(spark.sql(f"describe table extended catalog_databricks.schema_databricks.table_databricks_managed"))
display(spark.sql(f"describe table extended catalog_databricks.schema_databricks.table_databricks_external"))
display(spark.sql(f"describe volume catalog_databricks.schema_databricks.volume_databricks_managed"))
display(spark.sql(f"describe volume catalog_databricks.schema_databricks.volume_databricks_external"))



In [0]:
%sql
ALTER TABLE catalog_databricks.schema_databricks.table_databricks_external
ADD COLUMNS (EID INT, Ename STRING, DID INT);

insert into catalog_databricks.schema_databricks.table_databricks_external
select * from workspace.tablecreation.employee;

ALTER TABLE catalog_databricks.schema_databricks.table_databricks_managed
ADD COLUMNS (EID INT, Ename STRING, DID INT);

insert into catalog_databricks.schema_databricks.table_databricks_managed
select * from workspace.tablecreation.employee;

In [0]:
%python
display(spark.sql(f"select * from catalog_databricks.schema_databricks.table_databricks_external"));

In [0]:
%python
display(spark.sql("""
                  SELECT * 
FROM csv.`/Volumes/catalog_databricks/schema_databricks/volume_databricks_external/Table_Detail.csv`
with (header="true")
"""))

df=spark.read.format('csv').option("header","true").load("/Volumes/catalog_databricks/schema_databricks/volume_databricks_external/Table_Detail.csv")
display(df)

df.write.format("delta").option("path", "s3://s3bucket-databrickslearning/Table_Detail/").mode("overwrite").saveAsTable("catalog_databricks.schema_databricks.Table_Detail")
